In [1]:
# enabling 3rd party widgets
# from google.colab import output
# output.enable_custom_widget_manager()
# output.disable_custom_widget_manager()

# interactive 3D plot
# !pip install ipympl
# %matplotlib widget

In [2]:
import os
import math
import numpy as np
import matplotlib.pyplot as plt

import time as time
import platform as platform

import h5py

In [3]:
colab_flag = False

strategy = None
# strategy = tf.distribute.MirroredStrategy()

In [4]:
current_sys = platform.system()

if current_sys == 'Windows':
    dir_sep = '\\'
else:
    dir_sep = '/'

In [5]:
if colab_flag == True:
    from google.colab import drive
    drive.mount('/content/drive')
    os.chdir('/content/drive/MyDrive/Github/MLROM/KS/')

In [6]:
print(os.getcwd())

/home/rkaushik/Documents/Thesis/MLROM/new_lorenz


In [7]:
from tools.misc_tools import create_Lorenz_data, compute_lyapunov_spectrum

In [8]:
# setting seed for PRNGs
prng_seed = 42
np.random.seed(prng_seed)

FTYPE = np.float64
ITYPE = np.int64

# Lorenz System

In [9]:
rho_arr = [35, 45]# [20, 25, 30]
beta_arr = [4/3, 8/3]
sigma_arr = [10, 20]

params_mat = np.empty(shape=(len(rho_arr)*len(beta_arr)*len(sigma_arr), 3), dtype=FTYPE)
counter = 0
for i in range(len(sigma_arr)):
    for j in range(len(rho_arr)):
        for k in range(len(beta_arr)):
            sigma = sigma_arr[i]
            rho = rho_arr[j]
            beta = beta_arr[k]
            params_mat[counter, 0] = sigma
            params_mat[counter, 1] = rho
            params_mat[counter, 2] = beta
            chaos_cond = sigma + beta + 3
            chaos_cond /= sigma - beta - 1
            chaos_cond *= sigma
            chaos_cond = rho - chaos_cond
            if chaos_cond < 0:
                print('sigma : {:.2f}, rho : {:.2f}, beta : {:.2f} -- no chaos'.format(sigma, rho, beta))
            elif chaos_cond == 0.0:
                print('sigma : {:.2f}, rho : {:.2f}, beta : {:.2f} -- borderline chaos'.format(sigma, rho, beta))
            else:
                print('sigma : {:.2f}, rho : {:.2f}, beta : {:.2f} -- yes chaos'.format(sigma, rho, beta))
            counter += 1

sigma : 10.00, rho : 35.00, beta : 1.33 -- yes chaos
sigma : 10.00, rho : 35.00, beta : 2.67 -- yes chaos
sigma : 10.00, rho : 45.00, beta : 1.33 -- yes chaos
sigma : 10.00, rho : 45.00, beta : 2.67 -- yes chaos
sigma : 20.00, rho : 35.00, beta : 1.33 -- yes chaos
sigma : 20.00, rho : 35.00, beta : 2.67 -- yes chaos
sigma : 20.00, rho : 45.00, beta : 1.33 -- yes chaos
sigma : 20.00, rho : 45.00, beta : 2.67 -- yes chaos


In [10]:
# setting up params (and saving, if applicable)
print(params_mat)
num_cases = params_mat.shape[0]

t0 = 0.0
T = 600.0
delta_t = 0.01

return_params_arr = False
normalize_flag = False
alldata_withparams_flag = False

init_state = np.array([1, 1, 1], dtype=FTYPE)

[[10.         35.          1.33333333]
 [10.         35.          2.66666667]
 [10.         45.          1.33333333]
 [10.         45.          2.66666667]
 [20.         35.          1.33333333]
 [20.         35.          2.66666667]
 [20.         45.          1.33333333]
 [20.         45.          2.66666667]]


### Creating initial data, for transient inspection and cutoff

In [11]:
time_creation = time.time()
res_dict = create_Lorenz_data(
    T, t0, delta_t, 
    params_mat, init_state,
    return_params_arr=return_params_arr,
    normalize=normalize_flag,
    alldata_withparams=alldata_withparams_flag, FTYPE=FTYPE, ITYPE=ITYPE
)
time_creation = time.time() - time_creation
print('time_creation : {:.0f} s'.format(time_creation))

all_data = res_dict['all_data']
N = res_dict['N']
boundary_idx_arr = res_dict['boundary_idx_arr']
params_arr = res_dict['params_arr']
normalization_constant_arr = res_dict['normalization_constant_arr']

time_creation : 33 s


In [12]:
# making relevant helper arrays to cut off initial transient
initial_t0 = 500
T -= initial_t0


initial_t0 = np.array(
    [initial_t0]*params_mat.shape[0], dtype=FTYPE) # try to keep all entries the same
initial_idx = np.zeros_like(initial_t0, dtype=ITYPE)

begin_idx = 0
len_relevant_idx = 0
for i in range(len(initial_t0)):
    init_idx = ITYPE( (initial_t0[i] + 0.5*delta_t)//delta_t )
    initial_idx[i] = init_idx + begin_idx
    len_relevant_idx += boundary_idx_arr[i] - initial_idx[i]
    begin_idx = boundary_idx_arr[i]

relevant_idx = np.empty(shape=len_relevant_idx, dtype=ITYPE)
start = 0
for i in range(len(initial_t0)):
    end = start + (boundary_idx_arr[i] - initial_idx[i])
    relevant_idx[start:end] = np.arange(initial_idx[i], boundary_idx_arr[i])
    start = end

In [13]:
print('boundary_idx_arr :', boundary_idx_arr)
print('initial_idx :', initial_idx)

boundary_idx_arr : [ 60001 120002 180003 240004 300005 360006 420007 480008]
initial_idx : [ 50000 110001 170002 230003 290004 350005 410006 470007]


In [14]:
# cutting off initial transient
all_data = all_data[relevant_idx]
# init_idx = 0
# for i in range(len(boundary_idx_arr)):
#     init_idx += (initial_t0[i] + 0.25*delta_t)//delta_t
#     boundary_idx_arr[i] -= init_idx

prev_boundary_idx = 0
for i in range(num_cases):
    boundary_idx_arr[i] = prev_boundary_idx + (boundary_idx_arr[i] - initial_idx[i])
    prev_boundary_idx = boundary_idx_arr[i]

In [15]:
boundary_idx_arr

array([10001, 20002, 30003, 40004, 50005, 60006, 70007, 80008])

### Saving the data

In [16]:
# making ae save directory
dir_name_data = os.getcwd() + dir_sep + 'saved_data'
if not os.path.isdir(dir_name_data):
    os.makedirs(dir_name_data)

counter = 0
while True:
    dir_check = 'data_' + str(counter).zfill(3)
    if os.path.isdir(dir_name_data + dir_sep + dir_check):
        counter += 1
    else:
        break

dir_name_data = dir_name_data + dir_sep + dir_check
os.makedirs(dir_name_data)
print('dir_name_data:', dir_name_data)


# saving sim data
sim_data = {
    'params_mat':params_mat,
    't0':t0,
    'T':T,
    'delta_t':delta_t,
    'return_params_arr':return_params_arr,
    'normalize_flag':normalize_flag,
    'alldata_withparams_flag':alldata_withparams_flag,
    'prng_seed':prng_seed,
}
with open(dir_name_data+dir_sep+'sim_data_params.txt', 'w') as f:
    f.write(str(sim_data))

dir_name_data: /home/rkaushik/Documents/Thesis/MLROM/new_lorenz/saved_data/data_011


In [17]:
# saving the data
np.savez(
    dir_name_data+dir_sep+'data',
    all_data=all_data,
    boundary_idx_arr=boundary_idx_arr,
    normalization_constant_arr=[normalization_constant_arr],
    initial_t0=initial_t0,
    init_state_org=init_state,
    # init_state_mat=init_state_mat,
    # lyapunov_spectrum_mat=lpspectrum_mat,
    )

### Computing the Lyapunov spectrum

In [18]:
cdf_kwargs = {
    'T':500,
    't0':0,
    'delta_t':delta_t,
    'return_params_arr':False,
    'normalize':False,
    'alldata_withparams':False,
    'FTYPE':np.float64,
    'ITYPE':np.int64
}

create_data_fn = create_Lorenz_data
num_modes = 3

init_state_mat = np.empty(shape=(num_cases, num_modes), dtype=FTYPE)
begin_idx = 0
for i in range(num_cases):
    init_state_mat[i, :] = all_data[begin_idx, 0:num_modes]
    begin_idx = boundary_idx_arr[i]

dy = 1e-9
dy_mat = np.empty(shape=num_cases, dtype=FTYPE)
begin_idx = 0
for i in range(num_cases):
    # dy_mat[i] = dy * np.mean(  np.linalg.norm( all_data[begin_idx:boundary_idx_arr[i]], axis=1 )/xgrid.shape[0]  ) timeMean of spaceMeanofSpaceNorm
    # dy_mat[i] = dy * np.mean( np.linalg.norm( all_data[begin_idx:boundary_idx_arr[i], 0:num_modes], axis=0 )/( (boundary_idx_arr[i]-begin_idx)**0.5 ) ) # spaceMean of timeRMS
    dy_mat[i] = np.mean( np.mean(all_data[begin_idx:boundary_idx_arr[i], 0:num_modes]**2, axis=0)**0.5 )
    # print(dy_mat[i])
    dy_mat[i] *= dy
    begin_idx = boundary_idx_arr[i]
    print('dy_mat[{}] :'.format(i), dy_mat[i])
print('')

# cleaning up
del(all_data)
del(boundary_idx_arr)
del(params_arr)
del(normalization_constant_arr)
del(relevant_idx)
del(res_dict)

lpspectrum_mat, _ = compute_lyapunov_spectrum(
    create_data_fn, cdf_kwargs, num_modes, 
    init_state_mat, params_mat, dy_mat,
    zeta=10, delta_completionratio=0.1, num_exp=None, print_flag=True)

dy_mat[0] : 1.5596181693181222e-08
dy_mat[1] : 1.7194007917633028e-08
dy_mat[2] : 1.9642692109108038e-08
dy_mat[3] : 2.1589664299171166e-08
dy_mat[4] : 1.5314014655658193e-08
dy_mat[5] : 1.6942159103805054e-08
dy_mat[6] : 1.9275544997910122e-08
dy_mat[7] : 2.1318886267286237e-08

number of evaluation intervals per case: 5000

case 1 completion_ratio : 0.1, elapsed_time : 1.578s, global_completion : 1.25%
case 1 completion_ratio : 0.2, elapsed_time : 3.167s, global_completion : 2.5%
case 1 completion_ratio : 0.3, elapsed_time : 4.775s, global_completion : 3.75%
case 1 completion_ratio : 0.4, elapsed_time : 6.321s, global_completion : 5.0%
case 1 completion_ratio : 0.5, elapsed_time : 7.963s, global_completion : 6.25%
case 1 completion_ratio : 0.6, elapsed_time : 9.578s, global_completion : 7.5%
case 1 completion_ratio : 0.7, elapsed_time : 10.96s, global_completion : 8.75%
case 1 completion_ratio : 0.8, elapsed_time : 12.506s, global_completion : 10.0%
case 1 completion_ratio : 0.9, ela

In [19]:
lpspectrum_mat.sort(axis=1)
lpspectrum_mat = lpspectrum_mat[:, -1::-1]

In [20]:
print(lpspectrum_mat)

[[ 7.36138692e-01 -3.30291032e-03 -1.30660998e+01]
 [ 1.04036506e+00 -1.24485352e-03 -1.47056782e+01]
 [ 9.29783380e-01 -8.57869983e-04 -1.32621840e+01]
 [ 1.21286483e+00 -2.40770179e-03 -1.48770083e+01]
 [ 1.51698138e-03 -1.05745310e-02 -2.23230654e+01]
 [ 1.04422142e+00 -4.59426401e-03 -2.47047118e+01]
 [ 6.05984066e-01 -3.13855514e-03 -2.29347385e+01]
 [ 1.17621285e+00 -2.13240356e-03 -2.48389982e+01]]


In [21]:
KY_dim_lst = []

for i in range(lpspectrum_mat.shape[0]):
    sum_ = np.cumsum(lpspectrum_mat[i])
    KY_idx = np.where(sum_ >= 0.0)[0]
    KY_idx = np.max(KY_idx) + 1
    KY_dim = KY_idx + sum_[KY_idx-1]/np.abs(lpspectrum_mat[i, KY_idx])
    KY_dim_lst.append(KY_dim)
    
KY_dim_lst = np.array(KY_dim_lst)
print(KY_dim_lst)

[2.0560868  2.07066115 2.07004318 2.08136428 1.14345614 2.04208214
 2.02628526 2.04726762]


### Computing new data based on 'initial states' from the transient cutoffs and total time from mean Lyapunov time

In [22]:
with np.load(dir_name_data+dir_sep+'data.npz', allow_pickle=True) as fl:
    all_data = np.array(fl['all_data'])
    boundary_idx_arr = np.array(fl['boundary_idx_arr'])
    normalization_constant_arr = np.array(fl['normalization_constant_arr'])
    initial_t0 = np.array(fl['initial_t0'])
    init_state_org = np.array(fl['init_state_org'])

In [23]:
# saving the data
np.savez(
    dir_name_data+dir_sep+'data',
    all_data=all_data,
    boundary_idx_arr=boundary_idx_arr,
    normalization_constant_arr=normalization_constant_arr,
    initial_t0=initial_t0,
    init_state_org=init_state_org,
    init_state_mat=init_state_mat,
    lyapunov_spectrum_mat=lpspectrum_mat,
    KY_dim_arr=KY_dim_lst,
)

In [24]:
with open(dir_name_data+'/lp_exp.txt', 'w') as f:
    f.write(str({
        'lp_exp':list(lpspectrum_mat),
        'DKY_lst':list(KY_dim_lst),
        'lp_time':list(lpspectrum_mat[:, 0]**(-1))
    }))

In [25]:
# fl = np.load(dir_name_data+dir_sep+'data.npz', allow_pickle=True)

In [26]:
# fl.files